In [1]:
import pandas as pd
import numpy as np

# --- Master (like 'products') ---
stocks = pd.DataFrame({
    "ticker": ["ABC", "XYZ", "MNO", "HDP"],
    "company": ["Alpha Inc", "Xyza Corp", "Mono Labs", "HeadPro"],
    "sector": ["Tech", "Tech", "Healthcare", "Consumer"],
    "avg_cost": [120.0, 75.0, 50.0, 20.0],  # illustrative per-share book cost
})

# --- Daily prices (time series) ---
dates = pd.to_datetime([
    "2024-01-05","2024-01-10","2024-02-01","2024-02-20","2024-03-01","2024-03-15"
])
prices = pd.DataFrame({
    "date": np.repeat(dates, 4),
    "ticker": ["ABC","XYZ","MNO","HDP"] * len(dates),
    "close": [130,78,52,22,  128,79,54,21,  140,85,55,23,  145,82,53,24,  150,88,57,25,  155,90,56,26],
    "volume": [10000,12000,8000,7000,  9000,13000,7500,7200,  15000,16000,9000,8000,  14000,15500,8500,8200,  16000,17000,9500,8300,  15000,16500,9200,8400]
})

# --- Trade lines (each row is a trade fill, like order lines) ---
trades = pd.DataFrame({
    "trade_id": [1,1,2,3,3,4,5,5,6],
    "date": pd.to_datetime([
        "2024-01-05","2024-01-05","2024-01-10","2024-02-01","2024-02-01","2024-02-20","2024-03-01","2024-03-01","2024-03-15"
    ]),
    "ticker": ["ABC","HDP","XYZ","ABC","MNO","MNO","XYZ","HDP","ABC"],
    "side": ["SELL","SELL","BUY","SELL","SELL","BUY","SELL","SELL","SELL"],  # simple demo
    "price": [135,21,80,138,54,52,87,24,150],
    "quantity": [10,100,50,5,20,30,15,120,8],
    "broker": ["A","A","B","C","C","D","E","E","F"]
})

# --- Dividends (optional corporate action) ---
dividends = pd.DataFrame({
    "date": pd.to_datetime(["2024-02-15","2024-03-10"]),
    "ticker": ["ABC","XYZ"],
    "div_per_share": [1.0, 0.5]
})


In [2]:
# Revenue per trade line and per trade
trades = trades.copy()
trades["line_value"] = trades["price"] * trades["quantity"]

trade_value = (
    trades.groupby("trade_id", as_index=False)["line_value"]
          .sum()
          .rename(columns={"line_value": "trade_value"})
)
print(trade_value)


   trade_id  trade_value
0         1         3450
1         2         4000
2         3         1770
3         4         1560
4         5         4185
5         6         1200


In [3]:
# Join stock info (ticker -> company, sector, avg_cost)
trades_enriched = trades.merge(stocks, on="ticker", how="left")
print(trades_enriched[["trade_id","ticker","sector","price","quantity","avg_cost"]])


   trade_id ticker      sector  price  quantity  avg_cost
0         1    ABC        Tech    135        10     120.0
1         1    HDP    Consumer     21       100      20.0
2         2    XYZ        Tech     80        50      75.0
3         3    ABC        Tech    138         5     120.0
4         3    MNO  Healthcare     54        20      50.0
5         4    MNO  Healthcare     52        30      50.0
6         5    XYZ        Tech     87        15      75.0
7         5    HDP    Consumer     24       120      20.0
8         6    ABC        Tech    150         8     120.0


In [4]:
# Simple realized profit on SELL trades only (demo); BUY lines profit=0
trades_enriched["line_profit"] = np.where(
    trades_enriched["side"] == "SELL",
    (trades_enriched["price"] - trades_enriched["avg_cost"]) * trades_enriched["quantity"],
    0.0
)

profit_by_sector = (
    trades_enriched.groupby("sector", as_index=False)["line_profit"].sum()
)
print(profit_by_sector)


       sector  line_profit
0    Consumer        580.0
1  Healthcare         80.0
2        Tech        660.0


In [5]:
# Daily returns per ticker
prices_ts = prices.sort_values(["ticker","date"]).set_index("date")
prices_ts["daily_ret"] = (
    prices_ts.groupby("ticker")["close"].pct_change()
)

# Month-end prices and MoM returns per ticker (wide)
me_prices = (
    prices_ts.groupby("ticker")["close"]
             .resample("M")
             .last()
             .unstack(level=0)
)
monthly_returns = me_prices.pct_change().fillna(0.0)
print(monthly_returns)


ticker           ABC       HDP       MNO       XYZ
date                                              
2024-01-31  0.000000  0.000000  0.000000  0.000000
2024-02-29  0.132812  0.142857 -0.018519  0.037975
2024-03-31  0.068966  0.083333  0.056604  0.097561


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24616\2726450546.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


In [7]:
# Rolling indicators per ticker (e.g., 3-observation SMA / VOL for demo window)
roll = (
    prices_ts.groupby("ticker")[["close"]]
             .rolling(window=3, min_periods=1)
             .agg({"close": ["mean", "std"]})
             .rename(columns={"mean": "sma_3", "std": "vol_3"})
             .reset_index(level=0)
)
# Flatten MultiIndex columns
roll.columns = [col[0] if col[1]=="" else col[1] if col[0]=="close" else col[0] for col in roll.columns]
# Combine back with prices_ts
prices_with_roll = prices_ts.join(roll.set_index(prices_ts.index)[["sma_3","vol_3"]])
print(prices_with_roll[["ticker","close","sma_3","vol_3"]].head(12))


           ticker  close       sma_3     vol_3
date                                          
2024-01-05    ABC    130  130.000000       NaN
2024-01-05    ABC    130   22.000000       NaN
2024-01-05    ABC    130   52.000000       NaN
2024-01-05    ABC    130   78.000000       NaN
2024-01-10    ABC    128  129.000000  1.414214
2024-01-10    ABC    128   21.500000  0.707107
2024-01-10    ABC    128   53.000000  1.414214
2024-01-10    ABC    128   78.500000  0.707107
2024-02-01    ABC    140  132.666667  6.429101
2024-02-01    ABC    140   22.000000  1.000000
2024-02-01    ABC    140   53.666667  1.527525
2024-02-01    ABC    140   80.666667  3.785939


In [9]:
# Map ticker -> sector for sector-level aggregation
ticker_to_sector = stocks.set_index("ticker")["sector"]

# Monthly returns already in 'monthly_returns' (wide: columns=tickers)
monthly_returns_long = (
    monthly_returns.stack().rename("ret").reset_index()
    .rename(columns={"level_0":"date","level_1":"ticker"})
)

# Attach sector and compute equal-weight sector return each month
monthly_returns_long["sector"] = monthly_returns_long["ticker"].map(ticker_to_sector)
sector_monthly = (
    monthly_returns_long.groupby(["date","sector"], as_index=False)["ret"].mean()
    .sort_values(["date","sector"])
)
print(sector_monthly)


        date      sector       ret
0 2024-01-31    Consumer  0.000000
1 2024-01-31  Healthcare  0.000000
2 2024-01-31        Tech  0.000000
3 2024-02-29    Consumer  0.142857
4 2024-02-29  Healthcare -0.018519
5 2024-02-29        Tech  0.085394
6 2024-03-31    Consumer  0.083333
7 2024-03-31  Healthcare  0.056604
8 2024-03-31        Tech  0.083263


In [10]:
# Monthly dividend cash per ticker
div_monthly = (
    dividends.set_index("date")
             .groupby("ticker")["div_per_share"]
             .resample("M")
             .sum()
             .unstack(level=0)
             .fillna(0.0)
)
print(div_monthly)


ticker      ABC  XYZ
date                
2024-02-29  1.0  0.0
2024-03-31  0.0  0.5


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24616\4230240829.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")
